In [2]:
!pwd

/d/Education/Text_summarizer/Text_summarizer/research


In [3]:
import os 
os.chdir("../")

In [21]:
!pwd

/d/Education/Text_summarizer/Text_summarizer


In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path 
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [24]:
from textsummarizer.constants import * 
from textsummarizer.utils.common import read_yaml,create_directories

In [30]:
class ConfigurationManager: 
    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_file_path= PARAMS_FILE_PATH):

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig: 
        config = self.config.model_trainer 
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= config.num_train_epochs,
            warmup_steps= config.warmup_steps,
            per_device_train_batch_size= config.per_device_train_batch_size,
            per_device_eval_batch_size= config.per_device_eval_batch_size,
            weight_decay= config.weight_decay,
            logging_steps= config.logging_steps,
            evaluation_strategy= config.evaluation_strategy,
            eval_steps= config.eval_steps,
            save_steps= config.save_steps,
            gradient_accumulation_steps= config.gradient_accumulation_steps,
            
        )

        return model_trainer_config

In [31]:
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch 

In [27]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_ckpt = "google/pegasus-cnn_dailymail"
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
        seq2seq_data_collector= DataCollatorForSeq2Seq(tokenizer,model= model_pegasus)

        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(root_dir= self.config.root_dir,
            data_path= self.config.data_path,
            model_ckpt= self.config.model_ckpt,
            num_train_epochs= self.config.num_train_epochs,
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy,
            eval_steps= self.config.eval_steps,
            save_steps= self.config.save_steps,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps)

        # trainer_args = TrainingArguments(
        #                 output_dir='pegasus-samsum', warmup_steps=500,
        #                 per_device_train_batch_size=1, per_device_eval_batch_size=1,
        #                 weight_decay=0.01, logging_steps=10,
        #                 evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
        #                 gradient_accumulation_steps=16
        #             )
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collector=seq2seq_data_collector,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ##save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        ## save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        

        

In [28]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2023-07-07 12:04:40,392:INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-07 12:04:40,396:INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-07 12:04:40,397:INFO: common: created directory at: artifacts]
[2023-07-07 12:04:40,398:INFO: common: created directory at: artifacts/model_trainer]


BoxKeyError: "'ConfigBox' object has no attribute 'num_train_epochs'"